In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

from matplotlib import pyplot as plt

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_paths():
    """
    get lists of
    """

def make_5_fig_plots(path_list, sep_fact_list, title_save, Img_names, save_path = None, row_num = 3):
    """
    Create 5 panel plots of SSE vs SF
    """
    #Create a subplot
    fig, ax = create_subplots(len(bo_method_list))
    for i in range(len(bo_method_list)):
        title = "SF Analysis " + bo_method
        y_data, data_names = analyze_SF_data_for_plot(df, bo_method_list[i], sep_fact_list)
        # print(y_data)
        #Run Sep Fact Experiment Analysis (file_path)
        ax[i] = plot_SF_data(f, y_data, data_names, data_true, xbins, ybins, title, x_label, y_label, log_data, title_fontsize, 
                     other_fontsize, save_path)
    plt.show()



In [27]:
#Get all data from ep experiemnts
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# study_param_list = ["CONSTANT"]
study_param_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "ep"

#Get all ep_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [26]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 5]
study_id = "ep"
save_csv = False
cs_name_val = 1
project = signac.get_project()
df_best_all = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, meth_name_val, study_id, save_csv)
    df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
    df_best_all = pd.concat([df_best_all, df_best], ignore_index=True)
    
df_best_all

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
0,5,9,0.078748,0.444444,0.0,"[-0.6406910481352299, -1.25246266808837]",0.082193,0.035697,"[1.0265082505307273, -0.9709616321792267]",0.035697,"[1.0265082505307273, -0.9709616321792267]",0.163305,3,A1,10,1.633053,0.039318
1,7,1,1.675753,0.020273,0.0,"[-0.12469795110750015, 1.8446633771315404]",3.808574,0.326669,"[0.9078580839626224, -0.9156742903376899]",0.326669,"[0.9078580839626224, -0.9156742903376899]",0.209561,4,B1,2,0.419122,0.124904
2,1,34,0.0,2184164.409075,0.319077,"[1.0000110861663993, -0.9999857433378861]",0.0,0.0,"[1.0000088573535437, -0.9999856038393283]",0.0,"[1.0000088573535437, -0.9999856038393283]",12.663858,3,C2,35,443.235028,0.000017


In [15]:
#Get Best Data from ep experiment
save_csv = False
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method Val,BO Method,Max Evals,Total Run Time,L2 Norm Theta
124,7,27,7223.455785,1,0.0,"[-236.8421052631579, -31.57894736842107, -244....",362.033293,[1066.383104156883],"[-186.06214586285225, -132.38988417571917, -15...",[1066.383104156883],"[-186.06214586285225, -132.38988417571917, -15...",0.550877,CONSTANT,A1,28,15.424545,40.538549
251,1,22,8.885089,1,0.0,"[-257.89473684210526, -115.78947368421053, -17...",5904.047081,[555.3187326047051],"[-203.26528001480847, -99.90845232822555, -150...",[555.3187326047051],"[-203.26528001480847, -99.90845232822555, -150...",0.405823,CONSTANT,B1,24,9.73974,20.265362
691,6,2,0.002124,1,0.000351,"[-299.9999991942604, -52.631580535417406, -244...",0.000049,[0.0004865580806784452],"[-199.99953403288788, -100.013507056857, -170....",[0.0004865580806784452],"[-199.99953403288788, -100.013507056857, -170....",11.060329,CONSTANT,A2,5,55.301644,0.022950
1638,12,57,529.508596,1,13237.714895,"[-100.0, -5.128205128205138, -155.128205128205...",0.002066,[0.025122329047129483],"[-199.99998649710136, -99.86538595996633, -170...",[0.025122329047129483],"[-199.99998649710136, -99.86538595996633, -170...",860.527894,CONSTANT,B2,65,55934.313096,0.175561
1772,6,1,7223.455785,1,0.0,"[-300.0, 0.0, -250.0, 14.473684210526315]",0.001333,[4.679248462715612],"[-199.93357480418643, -99.45784188733653, -170...",[4.679248462715612],"[-199.93357480418643, -99.45784188733653, -170...",22.35226,CONSTANT,C2,2,44.70452,0.577030


In [ ]:
#Get Median Data from ep experiment
save_csv = False
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [ ]:
#Get Median of Best Data from ep experiment
save_csv = False
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best

In [2]:
#Get all data from sf experiemnts
date_time_str = "2023/09/06/"
# date_time_str = "No_Date/"
save_csv = False
name_cs_str = "CS1" 
meth_name_str_list = ["A1", "B1", "A2", "B2", "C2"]
# meth_name_str_list = ["C2"]
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# f = [0.9]
precision = 1
study_param_list = [f"{number:.{precision}f}" for number in f]
study_id = "sf"

#Get all sf_data  
df, theta_true = get_study_data_org(date_time_str, name_cs_str, meth_name_str_list, study_id, 
                                    study_param_list, save_csv = False)

In [7]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 3]
study_id = "sf"
save_csv = False
cs_name_val = 1
project = signac.get_project()
df_all = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, meth_name_val, study_id, save_csv)
    df_all = pd.concat([df_all, df], ignore_index=True)
    
# # Step 1: Filter the DataFrame based on values in column BO Method
# condition = (df_all['BO Method'] == 'C2')
# filtered_df = df_all[condition]

# # Step 2: Get unique values in column Sep Fact from the filtered DataFrame
# filtered_df['Sep Fact'].unique()

df = df_all

In [ ]:
#Get Best Data from ep experiment
df_best = get_best_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_best

In [ ]:
#Get Median Data from ep experiment
df_median = get_median_data(df, study_id, name_cs_str, theta_true, date_time_str, save_csv)

In [ ]:
#Get Median of Best Data from ep experiment
df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, date_time_str, save_csv)
df_median_best